In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/diabetes/description.pdf
/kaggle/input/diabetes/diabetic_data.csv
/kaggle/input/pima-indians-diabetes-database/diabetes.csv


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, accuracy_score, f1_score
import warnings
warnings.filterwarnings('ignore')

In [17]:
#==============================================================================
# DATASET 1: FOR READMISSION IN HOSPITALS 
#==============================================================================

print("\n\n" + "="*80)
print("Uploading diabetes 130 dataset")
print("="*80)

diabetes_one = pd.read_csv('/kaggle/input/diabetes/diabetic_data.csv')
print("step 1: Dataset loaded completed")
print("Shape right after loading:", diabetes_one.shape)



Uploading diabetes 130 dataset
step 1: Dataset loaded completed
Shape right after loading: (101766, 50)


In [ ]:
print("\nFeatures in dataset:")
for i, col in enumerate(diabetes_one.columns, 1):
    print(f"{i}.{col}")

calc_missing= (diabetes_one.isnull().sum() / len(diabetes_one) * 100)
drop_cols= calc_missing[calc_missing > 95].index
diabetes_one= diabetes_one.drop(columns=drop_cols)


diabetes_one['target'] = (diabetes_one['readmitted'] == '<30').astype(int)
print(f"  High Risk (readmitted <30 days): {(diabetes_one['target']==1).sum():,}")
print(f"  Low Risk: {(diabetes_one['target']==0).sum():,}")


In [19]:
print("\n[STEP 4] Feature Engineering...")

# Create 5 clinical features
if 'num_lab_procedures' in diabetes_one.columns:
    diabetes_one['had_lab_procedures']= (diabetes_one['num_lab_procedures']>0).astype(int)
if 'num_medications' in diabetes_one.columns:
    diabetes_one['high_medication_count'] = (diabetes_one['num_medications']> diabetes_one['num_medications'].median()).astype(int)
if 'number_diagnoses' in diabetes_one.columns:
    diabetes_one['multiple_diagnoses']=(diabetes_one['number_diagnoses']>5).astype(int)
if 'admission_type_id' in diabetes_one.columns:
    diabetes_one['is_emergency'] =(diabetes_one['admission_type_id']==1).astype(int)
if 'number_inpatient' in diabetes_one.columns:
    diabetes_one['had_inpatient_visits'] = (diabetes_one['number_inpatient']> 0).astype(int)



[STEP 4] Feature Engineering...


In [20]:
# Drop ID columns
diabetes_one= diabetes_one.drop(columns=['encounter_id', 'patient_nbr', 'readmitted'])

# Encode categorical columns
categorical_cols = diabetes_one.select_dtypes(include=['object']).columns.tolist()
for col in categorical_cols:
    le = LabelEncoder()
    diabetes_one[col] = le.fit_transform(diabetes_one[col].astype(str))

print(f"✓ Encoded {len(categorical_cols)} categorical columns")


✓ Encoded 36 categorical columns


In [22]:
print("\n[STEP 6] Training Random Forest Model...")

# Prepare data
X_diabetes_one=diabetes_one.drop('target', axis=1)
y_diabetes_one=diabetes_one['target']

# Train-test split
X_train_h,X_test_h,y_train_h,y_test_h= train_test_split(
    X_diabetes_one,y_diabetes_one,test_size=0.2,random_state=42,stratify=y_diabetes_one
)

print(f"✓ Training set: {len(X_train_h):,} | Test set: {len(X_test_h):,}")


[STEP 6] Training Random Forest Model...
✓ Training set: 81,412 | Test set: 20,354


In [24]:
# Simple parameter grid (fast training)
param_grid_hospital = {
    'n_estimators': [100, 200],
    'max_depth': [20, None],
    'min_samples_split': [2, 5]
}

# Train with GridSearch
rf_hospital = RandomForestClassifier(random_state=42, n_jobs=-1)
grid_hospital = GridSearchCV(rf_hospital, param_grid_hospital, cv=3, scoring='roc_auc', n_jobs=-1, verbose=0)

print("Training... (this takes 2-3 minutes)")
grid_hospital.fit(X_train_h, y_train_h)
# Best model
best_rf_hospital = grid_hospital.best_estimator_
print(f"\n✓ Best parameters: {grid_hospital.best_params_}")

# Predictions
y_pred_h = best_rf_hospital.predict(X_test_h)
y_proba_h = best_rf_hospital.predict_proba(X_test_h)[:, 1]

# Results
print("\n" + "-"*80)
print("PROJECT 1 RESULTS: Hospital Readmission Prediction")
print("-"*80)
print(f"Accuracy:  {accuracy_score(y_test_h, y_pred_h):.4f} ({accuracy_score(y_test_h, y_pred_h)*100:.2f}%)")
print(f"ROC-AUC:   {roc_auc_score(y_test_h, y_proba_h):.4f}")
print(f"F1-Score:  {f1_score(y_test_h, y_pred_h):.4f}")
print("\nClassification Report:")
print(classification_report(y_test_h, y_pred_h, target_names=['Low Risk', 'High Risk']))

# Feature importance
feature_names_h = X_diabetes_one.columns.tolist()
importance_h = best_rf_hospital.feature_importances_
importance_df_h = pd.DataFrame({
    'Feature': feature_names_h,
    'Importance': importance_h
}).sort_values('Importance', ascending=False)

print("\nTop 10 Risk Factors for Hospital Readmission:")
for i, row in importance_df_h.head(10).iterrows():
    print(f"  {i+1}. {row['Feature']}: {row['Importance']:.4f}")

Training... (this takes 2-3 minutes)

✓ Best parameters: {'max_depth': 20, 'min_samples_split': 5, 'n_estimators': 200}

--------------------------------------------------------------------------------
PROJECT 1 RESULTS: Hospital Readmission Prediction
--------------------------------------------------------------------------------
Accuracy:  0.8891 (88.91%)
ROC-AUC:   0.6642
F1-Score:  0.0157

Classification Report:
              precision    recall  f1-score   support

    Low Risk       0.89      1.00      0.94     18083
   High Risk       0.78      0.01      0.02      2271

    accuracy                           0.89     20354
   macro avg       0.84      0.50      0.48     20354
weighted avg       0.88      0.89      0.84     20354


Top 10 Risk Factors for Hospital Readmission:
  17. diag_1: 0.0822
  18. diag_2: 0.0800
  11. num_lab_procedures: 0.0782
  19. diag_3: 0.0776
  13. num_medications: 0.0657
  16. number_inpatient: 0.0551
  8. time_in_hospital: 0.0462
  6. discharge_dis

In [4]:
print("[STEP 1] Loading Pima Indians Dataset...")

diabetes_two = pd.read_csv("/kaggle/input/pima-indians-diabetes-database/diabetes.csv")

[STEP 1] Loading Pima Indians Dataset...
